For kicks, we're going to build a super simple neural network with only one layer.Structure
- Import libraries and data
- View shape
- Identify stocks we want to highlight
- These will be stocks that increase in value the next day
- So 'high' higher than previous 'high'
- Create a matrix of our training values
- Define our parameters matrix
- Pass data the data through activation function
- Pass to Loss function
- Back propagation

In [129]:
# First we import libraries and build arrays and tensors
import pandas as pd  # To manage the initial dataframe
import numpy as np  # Prec-process and organize the data
import tensorflow as tf  # Build tensors
import tensorflow.keras as keras  # Compute the activation and loss
from keras import activations

data_file = pd.read_csv('C:/Users/alexa/Documents/Code/Jupyter Playground/Apple Stocks/02-data/AppleStockPrices.csv')

data_file.head()

,Date,Open,High,Low,Close,Volume,Adj Close
0,1/3/2011,325.64,330.26,324.84,329.57,15883600,329.57
1,1/4/2011,332.44,332.50,328.15,331.29,11038600,331.29
2,1/5/2011,329.55,334.34,329.50,334.00,9058700,334.00
3,1/6/2011,334.72,335.25,332.90,333.73,10709500,333.73
4,1/7/2011,333.99,336.35,331.90,336.12,11096800,336.12


- We don't care about Date or Adj Close
- We will seperate out all those values where the next day is higher
- We will then have one set whose value after applying the sigmoid is 1
- We will also have another set whose value after applying the sigmoid is 0

In [130]:
# Here we build out the arrays and data structures that we need

# Identify the data we care about
wip_data = data_file[['Open', 'High', 'Low', 'Close', 'Volume']]  # The values we will use for the model

# Build our labeled df
High_stop = wip_data['High']  # Value we are testing against
High_start = wip_data['High'].shift(periods=1)  # Starting value to be subtracted

# Create a matrix with high subtracted from next days high
labeled_df = High_stop - High_start  # Values we will label

# We can then convert everything to numpy arrays to work with
ml_data = wip_data.to_numpy()  # Our primary data to numpy
ml_labels_wip = labeled_df.to_numpy()  # Our labels to numpy
ml_labels = np.empty(34, dtype=float)  # An empty array for our final labeling

# Convert the labels array into 0 and 1 labels
ml_labels[ml_labels_wip < 1] = 0   # Label values losing value
ml_labels[ml_labels_wip >= 1] = 1   # Label values gaining value

# Now we will build the data to train on
x_data_product = np.prod(ml_data, axis=1)  # Multiply rows in the main array so we have a single vector

# Normalize the multiplied data (min-max normalization)
min_val = np.min(x_data_product)  # Identify the smallest value
max_val = np.max(x_data_product)  # Identify the largest value
x = (x_data_product - min_val) / (max_val - min_val)  # Calculate the final normalized value

# Create the weights of our array
w = np.random.randn(len(x))  # Create a randomized set of weights for training

# Weighted array
ml_xw = x * w  # Multiply input values by weights (note that we're doing a row by row operation and not the dot product

print(ml_labels.shape)
print(ml_xw.shape)

(34,)
(34,)


In [131]:
# We're going to use the tanh function as our activation function
# We will use keras to do this as it does a lot of the work for us

# Convert our arrays to tensors
k_xw = tf.convert_to_tensor(ml_xw)
y = tf.convert_to_tensor(ml_labels)

# Run the activation function (Hyperbolic Tanh)
y_hat_init = keras.activations.tanh(k_xw)

# We're going to use binary cross entropy for our loss function so need to adjust our initial output
y_hat = (y_hat_init + 1) / 2.0
print(y_hat.shape)
print(y.shape)

(34,)
(34,)


In [133]:
# Binary cross entropy is defined as-(y\cdot{log}(p_i)+(1-y_i)\cdot{log}(1-p_i))
# We will do the operation manually in np

# Convert tensors to arrays
y_hat_np = tf.keras.ops.convert_to_numpy(y_hat)  # Convert y_hat to np array
y_np = tf.keras.ops.convert_to_numpy(y)  # Convert y to np array

# Removing the first value from each to get rid of the NaN value
y_hat_np = y_hat_np[1:] 
y_np = y_np[1:]
print(y_hat_np.shape)
print(y_np.shape)

bce = -np.mean((y_np * np.log(y_hat_np)) + (1 - y_np) * np.log(1 - y_hat_np))
print(bce)

(33,)
(33,)
0.7878855509684295


In [ ]:
# Next step will be to run backpropagation on the network to update the weights 

# The derivative of bce 
